In [ ]:
import numpy as np
import pandas as pd
import shap
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from joblib import load

from bd_lc_mediterranean.utilities.confusion_matrix import compute_confusion_matrix

### Data preprocessing

In [ ]:
data = pd.read_csv('dataset_postprocessed.csv')
data.dropna()
data = data[~data.isin([np.nan, np.inf, -np.inf]).any(1)]

In [ ]:
y_train_data = data["class"] 
x_train_data = data.drop(["class", "latitude", "longitude", "spring_product_name", "autumn_product_name", "summer_product_name"], axis=1)
pc_columns = x_train_data.columns

reduced_x_train_data = data[pc_columns]
reduced_x_train_data.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reduced_x_train_data, y_train_data, test_size=0.15, random_state=0,)
labels=y_train_data.unique()

In [ ]:
model = load('model.joblib')
y_true = model.predict(X_test)

In [ ]:
pred = pd.DataFrame(y_true).reset_index(drop=True, inplace=False)
real = y_test.reset_index(drop=True, inplace=False)
test = pd.DataFrame(X_test).reset_index(drop=True, inplace=False)
train = pd.DataFrame(X_train).reset_index(drop=True, inplace=False)

# Explainability with SHAP

In [ ]:
explainer = shap.TreeExplainer(model)

In [ ]:
X_test

In [ ]:
choosen_instance = X_test
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()

In [ ]:
shap.summary_plot(shap_values, X_train)

In [ ]:
labels

### Split by class

In [ ]:
def data_train(data, pc_columns):
    y_train_data = data["class"] 
    x_train_data = data.drop(["class", "latitude", "longitude", "spring_product_name", "autumn_product_name", "summer_product_name"], axis=1)

    reduced_x_train_data = data[pc_columns]
    #reduced_x_train_data.to_csv(f'{label}.csv')

    X_train, X_test, y_train, y_test = train_test_split(reduced_x_train_data, y_train_data, test_size=0.50, random_state=0,)

    y_true = model.predict(X_test)

    X_train = pd.DataFrame(X_train).reset_index(drop=True, inplace=False)
    X_test = pd.DataFrame(X_test).reset_index(drop=True, inplace=False)
    y_test = pd.DataFrame(y_test).reset_index(drop=True, inplace=False)
    y_train = pd.DataFrame(y_train).reset_index(drop=True, inplace=False)
    y_true = pd.DataFrame(y_true).reset_index(drop=True, inplace=False)
    

    return X_train, X_test, y_train, y_test, y_true


In [ ]:
labels = ['closedForest']

In [ ]:
for label in labels:
    b_aux = data['class'] == label
    b = data[b_aux]
    X_train, X_test, y_train, y_test, y_true = data_train(b, pc_columns)
    explainer = shap.TreeExplainer(model)
    choosen_instance = X_test.loc[0:10]
    shap_values = explainer.shap_values(choosen_instance)
    shap.initjs()
    shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)


In [ ]:
explainer = shap.TreeExplainer(model)
choosen_instance = X_test.loc[0:3]
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], choosen_instance)